# 回顾多层感知机
nn.Sequential定义了一种特殊的Module

In [1]:
import torch
from torch import nn
# 实现了大量常用函数
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0714, -0.1486, -0.1264,  0.1293,  0.0584, -0.1420,  0.1209,  0.0820,
         -0.1616,  0.0224],
        [ 0.0450, -0.1280, -0.0212,  0.0126,  0.1208, -0.0559,  0.1854,  0.1969,
         -0.2153,  0.0877]], grad_fn=<AddmmBackward0>)

自定义块

In [2]:
class MLP(nn.Module):
    # 定义需要哪些类，哪些参数
    def __init__(self):
        # 把需要的参数初始化
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[-0.4133,  0.2063,  0.0195,  0.3346, -0.0362, -0.0473,  0.1806, -0.1032,
         -0.0689, -0.0047],
        [-0.4467,  0.1642, -0.1121,  0.2790,  0.1003, -0.1825,  0.2045, -0.1047,
         -0.1199, -0.0327]], grad_fn=<AddmmBackward0>)

顺序块

In [8]:
# 功能与nn.Sequential几乎一致
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block  # 有序的
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-4.0628e-02,  9.9911e-02, -2.4939e-02, -4.8232e-02, -5.3178e-02,
         -1.7650e-01,  1.9488e-01, -4.7621e-02, -1.0651e-01, -2.1521e-01],
        [-1.6120e-02,  1.9196e-01,  5.0165e-02, -5.1662e-03, -7.5503e-02,
         -2.1303e-01,  2.0776e-01, -2.1733e-04, -1.3601e-01, -3.3363e-01]],
       grad_fn=<AddmmBackward0>)

在正向传播函数中执行代码

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)  # 不参加训练，因为不计算梯度
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()  # 返回一个标量

net = FixedHiddenMLP()
net(X)

tensor(-0.0187, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), 
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.5092, grad_fn=<SumBackward0>)

# 参数管理
首先关注具有单隐藏层的多层感知机

In [11]:
import torch 
from torch import nn

# nn.Linear(4, 8)是net[0]，nn.ReLU()是net[1]，nn.Linear(8, 1)是net[2]
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0968],
        [-0.1881]], grad_fn=<AddmmBackward0>)

参数访问

In [12]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.3307, -0.0955, -0.2742, -0.0908, -0.1257, -0.0118,  0.2500,  0.0279]])), ('bias', tensor([-0.0218]))])


目标参数

In [13]:
print(type(net[2].bias))
print(net[2].bias)  # 有梯度
print(net[2].bias.data)  # data指参数本身

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0218], requires_grad=True)
tensor([-0.0218])


In [14]:
net[2].weight.grad == None  # 未做反向计算

True

一次性访问所有参数

In [16]:
print(*[(name, param.shape) for name,param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [17]:
net.state_dict()['2.bias'].data

tensor([-0.0218])

从嵌套块收集参数

In [18]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):  # 插入4个block1
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.0595],
        [-0.0595]], grad_fn=<AddmmBackward0>)

看看已经设计好的网络是如何组织的

In [19]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


内置初始化

In [20]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)  # 下划线写在后面表示是一个替换函数
        nn.init.zeros_(m.bias)
        
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0141,  0.0011,  0.0124, -0.0107]), tensor(0.))

In [21]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块应用不同的初始化方法

In [23]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.6480,  0.5196, -0.3544,  0.0367])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


参数绑定

In [24]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100  # 共享权重，改了一个也会改另一个
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 自定义层
构造一个没有任何参数的自定义层

In [25]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

将层作为组件合并到构建更复杂的模型中

In [26]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-9.3132e-10, grad_fn=<MeanBackward0>)

带参数的图层

In [29]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.zeros(units, ))  
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-1.9707, -1.0354,  0.3036],
        [ 1.0072, -0.5514, -0.1807],
        [-0.3141,  0.4029,  0.7326],
        [-0.8407,  0.8867,  2.2052],
        [ 0.4488,  0.3725,  0.6554]], requires_grad=True)

使用自定义层直接执行正向传播计算

In [32]:
dense(torch.randn(2, 5))

tensor([[1.7455, 0.0000, 0.0000],
        [0.0000, 1.6883, 4.1752]])

使用自定义层构建模型

In [31]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

# 读写文件
加载和保存张量

In [33]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load("x-file")
x2

tensor([0, 1, 2, 3])

存储一个张量列表，然后把它们读回内存

In [34]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

写入或读取从字符串映射到张量的字典

In [35]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数

In [36]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
        
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

将模型的参数存储为一个叫做'mlp.params'的文件

In [37]:
torch.save(net.state_dict(), 'mlp.params')

实例化了原始多层感知机模型的一个备份。直接读取文件中存储的参数

In [38]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [40]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])